# New case studies for Robot Dance paper

In [ ]:
import os
import datetime
from importlib import reload
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pylab as plt

import run_robot
import prepare_data
reload(run_robot)

## Define Subnotification factor

Between 21st and 29th of July the city of São Paulo made public the result of a research that [17.9% of its population](https://www1.folha.uol.com.br/equilibrioesaude/2020/08/em-sao-paulo-22-dos-moradores-dos-bairros-mais-pobres-ja-pegaram-coronavirus.shtml) had alredy had Covid-19. Here we use that number to find out a reasonable subnotification factor.

In [ ]:
basic_prm = prepare_data.save_basic_parameters(min_level=0.8, rep=2.5, ndays=30)
subnot_factor = 11.6
cities_data = prepare_data.compute_initial_condition_evolve_and_save(basic_prm, "SP", ["Mun. São Paulo"], 10000000, subnot_factor, 1, "data/covid_with_drs_07_29.csv")
cities_data



Now, we define some important decisions:

* The basic reproduction rate (R0). The original literature and our own estimates suggest 2.5. But this value seems high nowdays when people are wearing masks, have learned stricter hygiene habits (more hand wahing), and do basic social distancing. I am trying now with 1.8.

* Horizon of simulation: we use a little more than one year because after that we should probably have a vacine and the game changes completely.

* What time series to use: we are using 7 days (which lead to larger ICU capacity).

* Lockdown level: what is the reproduction level achievable by a strict lockdown. We are using 0.8. Should be smaller than 1.

In [ ]:
# Define the basic data for the case studies

# Basic reproduction number
basic_rep = 1.8

# Simulation horizon
# A little more than a year when thevaccine should be here
ndays = 14*2*14

# Time series
need_icu = 0.00693521103887298
time_icu = 7
#need_icu = 0.00650997
#time_icu = 11

# Lockdown level
lock_level = 0.8

# Define basic paramters
basic_prm = prepare_data.save_basic_parameters(min_level=lock_level, rep=basic_rep, ndays=ndays, need_icu=need_icu, time_icu=time_icu)

# Compute initial values

# For cities
# cities_data = prepare_data.compute_initial_condition_evolve_and_save(basic_prm, "SP", ["Araçatuba", "São José Do Rio Preto"], 500000, 1)
# cities_data = prepare_data.compute_initial_condition_evolve_and_save(basic_prm, "SP", ["São José Do Rio Preto"], 25000, 6, 1)

# For DRS
cities_data = prepare_data.compute_initial_condition_evolve_and_save(basic_prm, "SP", [], 000000, subnot_factor, 1, "data/covid_with_drs_07_01.csv")

# Sub-groups for figures
sp = ["Mun. São Paulo"]
sp_so = sp + ["Sub região sudoeste - RMSP"]
rmsp = sp + ["Sub região leste - RMSP", "Sub região norte - RMSP", "Sub região oeste - RMSP", "Sub região sudeste - RMSP", "Sub região sudoeste - RMSP"]

In [ ]:
# Create a target matrix (max infected level)
ncities, ndays = len(cities_data.index), int(basic_prm["ndays"])
target = 0.8*np.ones((ncities, ndays))
target = prepare_data.save_target(cities_data, target)

# Use a forcedif that releases the cities in the end
force_dif = np.ones((ncities, ndays))
cities_data

In [ ]:
pd.set_option("display.width", 120)

# Simple function to run a test and save results
def run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, pools=None, verbosity=1):
    hammer_data = prepare_data.save_hammer_data(cities_data, 0, basic_prm["min_level"])
    run_robot.find_feasible_hammer(basic_prm, cities_data, M, target, hammer_data, out_file=None, 
        incr_all=True, verbosity=verbosity)
    run_robot.prepare_optimization(basic_prm, cities_data, M, target, hammer_data, force_dif, pools, verbosity=verbosity)
    run_robot.optimize_and_show_results(basic_prm, figure_file, result_file, cities_data, verbosity=verbosity)
    result = pd.read_csv(result_file, index_col=[0, 1])
    run_robot.plot_result(basic_prm, result, figure_file[:-4] + "_sp.png", hammer_data["duration"].values, 
        cities_data["start_date"][0], sp)
    plt.savefig(figure_file[:-4] + "_sp.png", dpi=150, bbox_inches='tight')
    run_robot.plot_result(basic_prm, result, figure_file[:-4] + "_spso.png", hammer_data["duration"].values, 
        cities_data["start_date"][0], sp_so)
    plt.savefig(figure_file[:-4] + "_sp_so.png", dpi=150, bbox_inches='tight')
    run_robot.plot_result(basic_prm, result, figure_file[:-4] + "_rmsp.png", hammer_data["duration"].values, 
        cities_data["start_date"][0], rmsp)
    plt.savefig(figure_file[:-4] + "_rmsp.png", dpi=150, bbox_inches='tight')


## Case 1: 14 day window, no alternation, no mobility

In [ ]:
# Define mobility matrix.
M = prepare_data.convert_mobility_matrix_and_save(cities_data, max_neighbors=0, drs=True)
M.loc["Mun. São Paulo", "Sub região sudoeste - RMSP"], M.loc["Sub região sudoeste - RMSP", "Mun. São Paulo"]

In [ ]:
%%time
basic_prm["alternate"] = 0.0
result_file = "results/window_14_noalt_nomobility.csv"
figure_file = "results/window_14_noalt_nomobility.png"
run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif)


## Case 2: 14 day window, no alternation, with mobility

In [ ]:
# Define mobility matrix (full connection)
M = prepare_data.convert_mobility_matrix_and_save(cities_data, max_neighbors=22, drs=True)
M.loc["Mun. São Paulo", "Sub região sudoeste - RMSP"], M.loc["Sub região sudoeste - RMSP", "Mun. São Paulo"]

In [ ]:
%%time
basic_prm["alternate"] = 0.0
result_file = "results/window_14_noalt_withmobility.csv"
figure_file = "results/window_14_noalt_withmobility.png"
run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif)

## Case 3: 14 day window, with alternation, with mobility

In [ ]:
# Start searching for when the "no alternation" solution decided for full opening.
results = pd.read_csv("results/window_14_noalt_withmobility.csv")
results = results[results["Variable"] == "rt"]
results.drop(["Variable"], axis=1, inplace=True)
results.set_index("City", inplace=True)

def find_last_opening(rts, rep):
    """Find the first moment where the decision of the nonalternating solution is
    to fully open the region.
    """
    rts = rts.values.copy()
    rts[rts < 0.95*rep] = 0.0
    return len(rts) - rts[::-1].argmin() + 1

# Turn off alternation after two windows after the time needed for opening.
for i in range(len(results.index)):
    opening = find_last_opening(results.iloc[i,:], basic_prm["rep"])
    force_dif[i, opening + 2*int(basic_prm["window"]):] = 0.0
    

In [ ]:
%%time
# Set up alternation weight
basic_prm["alternate"] = 1.0
result_file = "results/window_14_withalt_withmobility.csv"
figure_file = "results/window_14_withalt_withmobility.png"
run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif)

## Case 4: 14 day window, no alternation, with mobility, ICU shared in metropolitan area from day 1

In [ ]:
%%time
# Pool with all Sao Paulo metropolitan area
pools = [[1], [2], [3], [4], [5], [6], [7],[8], [9, 15, 16, 17, 18, 19], [10], [11], 
         [12], [13], [14], [20], [21], [22]]

force_dif =  np.ones((ncities, ndays))
basic_prm["alternate"] = 0.0
result_file = "results/window_14_noalt_withmobility_icushared.csv"
figure_file = "results/window_14_noalt_withmobility_icushared.png"
run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, pools)

## Some code to check results

In [ ]:
pool = np.array([19]) #np.array([9, 15, 16, 17, 18, 19]) - 1
cities_names = cities_data.iloc[pool].index
population = cities_data["population"]
icu_capacity = cities_data["icu_capacity"]
total_icus = np.array([(target.loc[c]*population.loc[c]*icu_capacity.loc[c]).values for c in cities_names]).sum(axis=0)
simulation = pd.read_csv("results/window_14_noalt_withmobility.csv", index_col=[0, 1])
#simulation = pd.read_csv("results/window_14_withalt_withmobility.csv", index_col=[0, 1])

first_day = 0 #hammer_data.iloc[pool, 0].min()
last_day = int(basic_prm["ndays"]) #first_day + 50 + 1
total_icus = total_icus[first_day:last_day]

if basic_prm["time_icu"] == 7:
    time_series_parameters = [0.00951258, 0.02407533, 0.01565422, 0.0, 1.04877035, -0.07470716, np.sqrt(0.00413135),
        0.01020326268631, 0.009768267929635]
    # time_series_parameters = [0.00926221, 0.01963079, 0.01034362, 0.0, 1.20388368, -0.22446884, np.sqrt(0.00495803),                    0.009882415226021, 0.010207401448971]
    # time_series_parameters =  [0.00683802, 0.02407533, 0.00483438, 0.0, 1.08987579, -0.09888466, np.sqrt(0.00210162), 
    # 0.007187944540946, 0.006838016532435]
elif basic_prm["time_icu"] == 11:
    time_series_parameters = [0.00643884, 0.01740896, 0.0112156, 0.0, 1.10547981, -0.1245054, np.sqrt(0.00368004),
        0.00692964502549, 0.006645629100376]
    # time_series_parameters = [0.00496131, 0.01740896, 0.0043539, 0.0, 1.12791817, -0.13644296, np.sqrt(0.00189557), 
    #     0.005290328560701, 0.004961305864535]
else:
    raise NotImplementedError

# Plot mean 
time_series = run_robot.SimpleTimeSeries(*time_series_parameters)
need_icu = [time_series.iterate(random=False) for i in range(int(basic_prm["ndays"]))]
used_icus = simulation.loc[cities_names[0], "i"]*need_icu*population[cities_names[0]]
for c in cities_names[1:]:
    used_icus += simulation.loc[c, "i"]*need_icu*population[c]
used_icus *= basic_prm["time_icu"]/basic_prm["tinf"]
used_icus = used_icus[first_day:last_day]
plt.plot(used_icus, color="C0", label="ICU occupation")

# Plot upper bound
p = 0.1
F1p = stats.norm.ppf(1.0 - p)
theta = np.array(time_series.theta).copy()
upper_bound = need_icu[:]
for j in range(len(upper_bound)):
    upper_bound[j] += F1p*time_series.sigmaw*time_series.delta*np.sqrt((theta[:j + 1]**2).sum())
used_icus = simulation.loc[cities_names[0], "i"]*upper_bound*population[cities_names[0]]
for c in cities_names[1:]:
    used_icus += simulation.loc[c, "i"]*upper_bound*population[c]
used_icus *= basic_prm["time_icu"]/basic_prm["tinf"]
used_icus = used_icus[first_day:last_day]
plt.plot(used_icus, label="", color="C0")

# Make random simulations
total_days = 0
bad_days = 0
for i in range(100):
    time_series.reset()
    total_days += last_day - first_day
    need_icu = [time_series.iterate(random=True) for i in range(int(basic_prm["ndays"]))]
    used_icus = simulation.loc[cities_names[0], "i"]*need_icu*population[cities_names[0]]
    for c in cities_names[1:]:
        used_icus += simulation.loc[c, "i"]*need_icu*population[c]
    used_icus *= basic_prm["time_icu"]/basic_prm["tinf"]
    used_icus = used_icus[first_day:last_day]
    bad_days += (used_icus > total_icus).sum()
    plt.plot(used_icus, label="", alpha=0.2, color="C0")

print(f"Bad days = {bad_days:d}/{total_days:d} == {bad_days / total_days * 100:f}%")

# Plot results
import matplotlib.pylab as plt
plt.plot(total_icus, color="C3", label="Maximal ICU target")
start_date = pd.Timestamp(cities_data["start_date"][0]) + first_day*pd.to_timedelta("1D")
ndays = len(simulation.loc[cities_names[0], "i"])
ticks = pd.date_range(start_date, start_date + (last_day - first_day)*pd.to_timedelta("1D"), freq="1MS")
ticks = list(ticks)
if ticks[0] <= start_date + pd.to_timedelta("10D"):
    ticks[0] = start_date
else:
    ticks = [start_date] + ticks
plt.gca().set_xticks([(i - start_date).days for i in ticks])
labels = [i.strftime('%d/%m/%Y') for i in ticks]
plt.gca().set_xticklabels(labels, rotation=45, ha='right')
plt.legend()
plt.savefig("results/icu-usage.png", dpi=150, bbox_inches='tight')



## Scratch area, you can ignore

In [ ]:

reload(run_robot)
from scipy.stats import norm
duration = 40
if basic_prm["time_icu"] == 7:
    time_series_data = [0.00951258, 0.02407533, 0.01565422, 0.0, 1.04877035, -0.07470716, np.sqrt(0.00413135),
        0.01020326268631, 0.009768267929635]
elif basic_prm["time_icu"] == 11:
    time_series_data = [0.00643884, 0.01740896, 0.0112156, 0.0, 1.10547981, -0.1245054, np.sqrt(0.00368004),
        0.00692964502549, 0.006645629100376]
else:
    raise NotImplementedError

time_series = run_robot.SimpleTimeSeries(*time_series_data)
eicu = [time_series.iterate() for i in range(duration)]
p = 0.1
F1p = norm.ppf(1.0 - p)
theta = np.array(time_series.theta).copy()
upper_bound = eicu[:]
for i in range(duration):
    upper_bound[i] += F1p*time_series.sigmaw*time_series.delta*np.sqrt((theta[:i + 1]**2).sum())
sample_icu = []
nsamples = 0
nOK = 0
for i in range(1000):
    start, stop = 0, duration + 1
    time_series.reset()
    samples = np.array([time_series.iterate(random=True) for i in range(duration)])
    sample_icu.append(samples)
    nsamples += len(samples[start:stop])
    nOK += (samples[start:stop] <= upper_bound[start:stop]).sum()

print(nOK, nsamples, 100*nOK/nsamples)

In [ ]:
reload(run_robot)
duration = 365
constant_fix = [0.00679005, 0.10136368, 0.00586580, 0.0, 1.97210897, -0.97759272, np.sqrt(0.00051777), 
    0.007307625663244, 0.007008117960396]
constant_trend_fix = [0.00679005, 0.10136368, 0.02727935, -0.00126350, 0.52217732, 0.42334116, np.sqrt(0.00656118),      
    0.007307625663244, 0.007008117960396]
trend_fix = [0.00679005, 0.10136368, 0.0, 0.00004587, 1.96641774, -0.96871210, np.sqrt(0.00057691),
    0.007307625663244, 0.007008117960396]
old = [0.00379873, 0.02360889, 0.003055220184503005, 0.0, 1.346540496346441, -0.35212183634836325, np.sqrt(0.0011820962652620602),
    0.009821952043627, 0.009099877277162]
    
constant_fix_new = [0.00679005, 0.01835854, 0.0112156, 0.0, 1.10547981, -0.1245054, np.sqrt(0.00368004), 
    0.007307625663244, 0.007008117960396]
trend_fix_new = [0.00679005, 0.01835854, 0.0, -0.00035806, 1.08276406, -0.0860476, np.sqrt(0.0036272),
    0.007307625663244, 0.007008117960396]
constant_trend_fix_new = [0.00679005, 0.01835854, 0.35433314, -0.00937098, 0.84891778, -0.25056312, np.sqrt(0.00294438),
    0.007307625663244, 0.007008117960396]

constant_7 = [0.01003145, 0.02538853, 0.0156431, 0.0, 1.04906141, -0.07498674, np.sqrt(0.00412906),
    0.007307625663244, 0.007008117960396]


real_data = [0.009653179181505, 0.009523875134098, 0.009444887860431, 0.009262212993882, 0.009478777174599, 0.010093499807287, 0.010179820005892, 0.010292394029863, 0.010528625867502, 0.010474377255532, 0.00995980626524, 0.009882852156456, 0.010547373988112, 0.010420737340341, 0.010247061232653, 0.01000847542721, 0.010060900166336, 0.009882415226021, 0.010207401448971, 0.011062869458313, 0.011366884433894, 0.011588585778359, 0.011596446414382, 0.011190661353815, 0.010490634335358, 0.009821952043627, 0.009099877277162, 0.007985224087466, 0.007374028710215, 0.007187944540946, 0.006838016532435, 0.006577043144098]

time_series_data_7 = [0.00951258, 0.02407533, 0.01565422, 0.0, 1.04877035, -0.07470716, np.sqrt(0.00413135),
    0.01020326268631, 0.009768267929635]
time_series_data_11 = [0.00643884, 0.01740896, 0.0112156, 0.0, 1.10547981, -0.1245054, np.sqrt(0.00368004),
    0.00692964502549, 0.006645629100376]

new_7 = [0.00926221, 0.01963079, 0.01034362, 0.0, 1.20388368, -0.22446884, np.sqrt(0.00495803), 0.009882415226021,
    0.010207401448971]
new_7 = [0.00683802, 0.02407533, 0.00483438, 0.0, 1.08987579, -0.09888466, np.sqrt(0.00210162), 
    0.007187944540946, 0.006838016532435]
new_11 = [0.00496131, 0.01740896, 0.0043539, 0.0, 1.12791817, -0.13644296, np.sqrt(0.00189557), 
    0.005290328560701, 0.004961305864535]

time_series_data = time_series_data_7
time_series = run_robot.SimpleTimeSeries(*time_series_data_11)
eicu = [time_series.iterate() for i in range(duration)]
plt.plot(eicu)
plt.plot(real_data)
for i in range(100):
    time_series.reset()
    random_traj = [time_series.iterate(random=True) for i in range(duration)]
    plt.plot(random_traj, color="C0", alpha=0.1)


time_series = run_robot.SimpleTimeSeries(*new_11)
eicu = [time_series.iterate() for i in range(duration)]
plt.plot(eicu, color="C1")
plt.plot(real_data)
for i in range(100):
    time_series.reset()
    random_traj = [time_series.iterate(random=True) for i in range(duration)]
    plt.plot(random_traj, color="C1", alpha=0.1)
